### ME3:

In [1]:
# Standard library imports
import time
from collections import defaultdict
from functools import wraps
from pathlib import Path
from typing import Tuple

# Third-party imports
import lightning as L
import torch
from einops import einsum
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision.models import ResNet18_Weights, resnet18, ResNet
from torch import nn
from torchmetrics.classification import Accuracy
from torch.nn.utils import prune
from tqdm.auto import tqdm

# set precision for matmuls
torch.set_float32_matmul_precision("medium")   # or "medium"

/home/aleisley/Documents/ai231/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
weights_and_biases = ResNet18_Weights.DEFAULT.get_state_dict()
print(weights_and_biases.keys())

odict_keys(['conv1.weight', 'bn1.running_mean', 'bn1.running_var', 'bn1.weight', 'bn1.bias', 'layer1.0.conv1.weight', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.conv2.weight', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.1.conv1.weight', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.conv2.weight', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer2.0.conv1.weight', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.conv2.weight', 'layer2.0.bn2.running_mean', 'layer2.0.bn2.running_var', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', 'layer2.0.downsample.0.weight', 'layer2.0.downsample.1.running_mean', 'layer2.0.downsample.1.running_var', 'layer2.0.downsample.1.weight', 'layer2.0.do

In [3]:
weights_and_biases['fc.weight']

Parameter containing:
tensor([[-0.0185, -0.0705, -0.0518,  ..., -0.0390,  0.1735, -0.0410],
        [-0.0818, -0.0944,  0.0174,  ...,  0.2028, -0.0248,  0.0372],
        [-0.0332, -0.0566, -0.0242,  ..., -0.0344, -0.0227,  0.0197],
        ...,
        [-0.0103,  0.0033, -0.0359,  ..., -0.0279, -0.0115,  0.0128],
        [-0.0359, -0.0353, -0.0296,  ..., -0.0330, -0.0110, -0.0513],
        [ 0.0021, -0.0248, -0.0829,  ...,  0.0417, -0.0500,  0.0663]],
       requires_grad=True)

#### Load the Datasets

In [4]:
transform = ResNet18_Weights.IMAGENET1K_V1.transforms()

# Build the full training set once, then carve out a validation split
full_train = CIFAR10(
    root="data/CIFAR10",
    train=True,
    transform=transform,
    download=True,
)

generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(full_train, [45000, 5000], generator=generator)

cifar10_test = CIFAR10(
    root="data/CIFAR10",
    train=False,
    transform=transform,
    download=True,
)

train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
    drop_last=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
)

test_loader = DataLoader(
    cifar10_test,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
)



#### Initialize the model and replace head

In [3]:
from typing import Tuple
import torch
import torch.nn as nn
import pytorch_lightning as L
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.models import resnet18, ResNet18_Weights
from torchmetrics.classification import Accuracy

class LitResnet18(L.LightningModule):
    def __init__(self, num_classes: int = 10, lr: float = 1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.model.fc = nn.Linear(512, num_classes)

        # freeze backbone
        for name, p in self.model.named_parameters():
            if not name.startswith("fc"):
                p.requires_grad = False

        self.loss = nn.CrossEntropyLoss()
        self.val_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.train_acc = Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

    def _shared_step(self, batch, batch_idx) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, y

    def training_step(self, batch, batch_idx):
        loss, preds, y = self._shared_step(batch, batch_idx)
        self.train_acc.update(preds, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("train_acc", self.train_acc, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, preds, y = self._shared_step(batch, batch_idx)
        self.val_acc.update(preds, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("val_acc", self.val_acc, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)

    def test_step(self, batch, batch_idx):
        loss, preds, y = self._shared_step(batch, batch_idx)
        self.test_acc.update(preds, y)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("test_acc", self.test_acc, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)

    def configure_optimizers(self):
        trainable = (p for p in self.parameters() if p.requires_grad)
        optimizer = torch.optim.SGD(
            trainable, lr=self.hparams.lr, momentum=0.9, weight_decay=1e-4
        )
        scheduler = MultiStepLR(optimizer, milestones=[30, 60], gamma=0.1)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch",
                "frequency": 1,
            }
        }


In [4]:
class CIFAR10DataModule(L.LightningDataModule):
    def __init__(
            self, 
            data_dir: str = "data/CIFAR10", 
            batch_size: int = 256, 
            num_workers: int = 0
        ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = ResNet18_Weights.IMAGENET1K_V1.transforms()

    def setup(self, stage: str | None = None):
        self.generator = torch.Generator().manual_seed(42)

        if stage in (None, 'fit', 'validate'):
            cifar10_full = CIFAR10(
                root=self.data_dir,
                train=True,
                transform=self.transform,
                download=True,
            )
            self.cifar10_train, self.cifar10_val = random_split(
                cifar10_full, [45000, 5000], generator=self.generator
            )
        
        if stage in (None, "test", "predict"):
            self.cifar10_test = CIFAR10(
                root=self.data_dir,
                train=False,
                transform=self.transform,
                download=True,
            )

    def train_dataloader(self):
        return DataLoader(
            self.cifar10_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            persistent_workers=False,
            pin_memory=True,
            drop_last=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.cifar10_val,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True,
            persistent_workers=False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.cifar10_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
            persistent_workers=self.num_workers > 0,
        )


In [25]:
model = LitResnet18()
cifar10_dm = CIFAR10DataModule(num_workers=0)

trainer = L.Trainer(max_epochs=90)
trainer.fit(model, datamodule=cifar10_dm)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | ResNet             | 11.2 M | train
1 | loss      | CrossEntropyLoss   | 0      | train
2 | val_acc   | MulticlassAccuracy | 0      | train
3 | test_acc  | MulticlassAccuracy | 0      | train
4 | train_acc | MulticlassAccuracy | 0      | train
---------------------------------------------------------
5.1 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
72        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/aleisley/Documents/ai231/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/aleisley/Documents/ai231/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 89: 100%|██████████| 175/175 [01:55<00:00,  1.52it/s, v_num=10, train_loss_step=0.558, val_loss=0.639, val_acc=0.778, train_loss_epoch=0.606, train_acc=0.793]

`Trainer.fit` stopped: `max_epochs=90` reached.


Epoch 89: 100%|██████████| 175/175 [01:55<00:00,  1.52it/s, v_num=10, train_loss_step=0.558, val_loss=0.639, val_acc=0.778, train_loss_epoch=0.606, train_acc=0.793]


In [18]:
trainer.save_checkpoint("resnet18_finetuned.ckpt")

# Reload the module later
# model = LitResnet18.load_from_checkpoint("resnet18_cifar10.ckpt")

In [8]:
trainer.callback_metrics

{'train_loss': tensor(0.8721),
 'train_loss_step': tensor(0.7433),
 'val_loss': tensor(0.8424),
 'val_acc': tensor(0.7294),
 'train_loss_epoch': tensor(0.8721),
 'train_acc': tensor(0.7277)}

In [9]:
trainer.callback_metrics["val_acc"].item()

0.7293999791145325

In [ ]:
per_cycle_amount = 0.05
cur_cycle_amount = 0.00

while cur_cycle_amount < 1.05:
    print(f"Pruning {cur_cycle_amount:.2f} of weights")
    cur_cycle_amount += per_cycle_amount

Pruning 0.00 of weights
Pruning 0.05 of weights
Pruning 0.10 of weights
Pruning 0.15 of weights
Pruning 0.20 of weights
Pruning 0.25 of weights
Pruning 0.30 of weights
Pruning 0.35 of weights
Pruning 0.40 of weights
Pruning 0.45 of weights
Pruning 0.50 of weights
Pruning 0.55 of weights
Pruning 0.60 of weights
Pruning 0.65 of weights
Pruning 0.70 of weights
Pruning 0.75 of weights
Pruning 0.80 of weights
Pruning 0.85 of weights
Pruning 0.90 of weights
Pruning 0.95 of weights
Pruning 1.00 of weights


In [ ]:
for cycle in range(num_cycles):
    trainer.fit(model, datamodule)
    acc = trainer.callback_metrics["val_acc"].item()
    results.append({"cycle": cycle, "val_acc": acc})

In [5]:
model = resnet18(weights='IMAGENET1K_V1')
model.fc = nn.Linear(in_features=512, out_features=10)

for name, param in model.named_parameters():
    if not name.startswith('fc'):
        param.requires_grad = False

In [6]:
non_head_trainable = [name for name, param in model.named_parameters() if param.requires_grad and not name.startswith('fc')]
assert not non_head_trainable, f'Non-head parameters still trainable: {non_head_trainable}'
print('Backbone frozen; only fc parameters require gradients.')


Backbone frozen; only fc parameters require gradients.


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    model.fc.parameters(),
    lr=0.01,
    momentum=0.9,
    weight_decay=1e-4,
    nesterov=True,
)

print('Setup criterion and optimizer for head-only fine-tuning.')


Setup criterion and optimizer for head-only fine-tuning.
